<a href="https://colab.research.google.com/github/Az-Ks/AirQo-Ugandan-Air-Quality-Forecast/blob/master/submission1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle 

In [2]:
#upload the credentials of the kaggle account
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"azerksouri","key":"b15ca1759cf75c7a18339ae387de7ba1"}'}

In [4]:
#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d azerksouri/airqo-ugandan-air-quality-forecast-challenge-zindi

 96% 39.0M/40.7M [00:01<00:00, 10.9MB/s]
100% 40.7M/40.7M [00:01<00:00, 31.2MB/s]


In [6]:
!unzip "airqo-ugandan-air-quality-forecast-challenge-zindi.zip"

Archive:  airqo-ugandan-air-quality-forecast-challenge-zindi.zip
  inflating: Test (1).csv            
  inflating: Train (1).csv           
  inflating: airqo_metadata.csv      
  inflating: sample_sub.csv          


In [7]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

In [8]:
# ! cp data/Train_target.csv ./raw_data/Train_target.csv
# ! cp data/Test_target.csv ./raw_data/Test_target.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
train=pd.read_csv("/content/Train (1).csv")
test=pd.read_csv("/content/Test (1).csv")

In [40]:
train.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",45.126304
1,ID_train_1,D,"22.53333333,21.71666667,20.83333333,20.9833333...","0.102,0.0,0.0,0.0,0.0,0.0,0.0,0.034,0.017,0.01...","0.744583333,0.808083333,0.911166667,0.91633333...","281.6643101,89.15629262,81.96853891,291.018632...","2.3775,1.126666667,0.700833333,0.3416666670000...","90.32,90.3775,90.44083333,90.4725,90.45416667,...",79.131702
2,ID_train_10,A,"28.975,27.95,29.6,26.425,22.09166667,21.775,22...","0.0,0.0,0.0,0.102,0.136,0.0,0.0,2.16,1.276,0.0...","0.573333333,0.597166667,0.5668333329999999,0.6...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","88.55166667,88.46416667,88.31916667,88.24,88.2...",32.661304
3,ID_train_100,A,"22.96666667,24.26666667,25.275,25.625,25.86666...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.77,3.012,1.0...","0.8430833329999999,0.79025,0.7375,0.728,0.7049...","300.0850574,293.6769595,294.5174647,301.921416...","1.446666667,1.1925,1.324166667,1.5441666669999...","88.615,88.53083333,88.4,88.27166667,88.2075,88...",53.850238
4,ID_train_1000,A,"21.875,21.575,21.525,21.43333333,20.50833333,1...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.8564166670000001,0.874916667,0.879833333,0.8...","21.83997432,17.05405341,89.26406044,123.585424...","0.1975,0.244166667,0.411666667,0.56,0.5775,0.4...","88.55666667,88.64083333,88.65833333,88.6475,88...",177.418750


In [41]:
test.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press
0,ID_test_0,C,"23.96666667,22.88333333,22.28333333,21.4833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.6898333329999999,0.722333333,0.7755833329999...","290.1352379,328.47011289999995,323.5730796,282...","0.6333333329999999,0.725,0.955,0.8725,0.6925,0...","87.64333333,87.67,87.70583333,87.75083333,87.7..."
1,ID_test_1,C,"18.99166667,19.70833333,20.95833333,22.7833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.99975,0.99725,0.956333333,0.851583333,0.7775...","295.2350543,307.6569135,313.7360236,311.518385...","0.640833333,1.244166667,1.284166667,1.2875,1.3...","87.8925,87.95666667,87.9925,87.9925,87.9808333..."
2,ID_test_10,D,"25.11666667,25.08333333,24.40833333,23.575,22....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.760166667,0.733583333,0.78475,0.825833333,0....","77.06161145,95.48124997,61.44498258,89.4688360...","2.0133333330000003,1.3025,0.783333333,0.530833...","90.34,90.2975,90.305,90.35583333,90.4425,90.49..."
3,ID_test_100,C,"18.71666667,20.60833333,22.28333333,23.9833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.604,0.0,0.0,...","0.951416667,0.92575,0.8603333329999999,0.77883...","310.7509058,316.186793,324.8167326,327.2174992...","0.745,1.169166667,1.316666667,1.385,1.40083333...","87.98833333,88.06333333,88.07833333,88.0366666..."
4,ID_test_1000,D,"17.63333333,18.05833333,19.625,21.28333333,23....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.9,0.898333333,0.8725,0.831666667,0.820833332...","334.6054069,329.6848561000001,322.7668036,312....","0.620833333,0.5,0.456666667,0.558333333,1.0158...","90.515,90.5575,90.62333333,90.6825,90.67916667..."


In [42]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])    

In [43]:
train.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",45.126304
1,ID_train_1,D,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[281.6643101, 89.15629262, 81.96853891, 291.01...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...",79.131702
2,ID_train_10,A,"[28.975, 27.95, 29.6, 26.425, 22.09166667, 21....","[0.0, 0.0, 0.0, 0.102, 0.136, 0.0, 0.0, 2.16, ...","[0.573333333, 0.597166667, 0.5668333329999999,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[88.55166667, 88.46416667, 88.31916667, 88.24,...",32.661304
3,ID_train_100,A,"[22.96666667, 24.26666667, 25.275, 25.625, 25....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.77,...","[0.8430833329999999, 0.79025, 0.7375, 0.728, 0...","[300.0850574, 293.6769595, 294.5174647, 301.92...","[1.446666667, 1.1925, 1.324166667, 1.544166666...","[88.615, 88.53083333, 88.4, 88.27166667, 88.20...",53.850238
4,ID_train_1000,A,"[21.875, 21.575, 21.525, 21.43333333, 20.50833...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.8564166670000001, 0.874916667, 0.879833333,...","[21.83997432, 17.05405341, 89.26406044, 123.58...","[0.1975, 0.244166667, 0.411666667, 0.56, 0.577...","[88.55666667, 88.64083333, 88.65833333, 88.647...",177.418750


### Features engineering part

In [44]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    #x["var_"+col_name]=x[col_name].apply(np.var)
    x["sum_"+col_name]=x[col_name].apply(np.sum)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]

In [45]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data.columns.tolist()

['ID',
 'location',
 'temp',
 'precip',
 'rel_humidity',
 'wind_dir',
 'wind_spd',
 'atmos_press',
 'target']

In [46]:
for x in range(121):
    data["newtemp"+ str(x)] = data.temp.str[x]
    data["newprecip"+ str(x)] = data.precip.str[x]
    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
    data["windspeed"+ str(x)] = data.wind_spd.str[x]
    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]

In [47]:
data.shape

(20574, 735)

In [48]:
len(data.precip[1])

121

In [49]:
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

100%|██████████| 6/6 [00:02<00:00,  2.75it/s]


In [50]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

100%|██████████| 6/6 [00:14<00:00,  2.36s/it]


In [51]:
data.drop(features,1,inplace=True)

In [52]:

hum_features =  list(data.filter(regex='rel_humidity.*').columns)
temp_features = list( data.filter(regex='newtemp.*').columns)  
precip_features =  list(data.filter(regex='newprecip*').columns)
winddir_features = list( data.filter(regex='newwind_dir.*').columns)
windspead_features = list( data.filter(regex='windspeed.*').columns)
atm_features =  list(data.filter(regex='atmos.*').columns)




hum_features= hum_features[36:]
temp_features=temp_features[36:] 
precip_features=precip_features[31:]
winddir_features=winddir_features[36:]
windspead_features=windspead_features[36:]
atm_features=atm_features[36:]





data[hum_features]= data[hum_features].apply(lambda x: x.fillna(x.mean()),axis=1)



data[temp_features]= data[temp_features].apply(lambda x: x.fillna(x.mean()),axis=1)


data[precip_features]= data[precip_features].apply(lambda x: x.fillna(float(0.0)),axis=1)



data[winddir_features]= data[winddir_features].apply(lambda x: x.fillna(x.mean()),axis=1)



data[windspead_features]= data[windspead_features].apply(lambda x: x.fillna(x.mean()),axis=1)




data[atm_features]= data[atm_features].apply(lambda x: x.fillna(x.mean()),axis=1)


In [53]:
train=data[data.target.notnull()].reset_index(drop=True)
test=data[data.target.isna()].reset_index(drop=True)

In [54]:
train.head()

,ID,location,target,newtemp0,newprecip0,newrel_humidity0,newwind_dir0,windspeed0,atmospherepressure0,newtemp1,newprecip1,newrel_humidity1,newwind_dir1,windspeed1,atmospherepressure1,newtemp2,newprecip2,newrel_humidity2,newwind_dir2,windspeed2,atmospherepressure2,newtemp3,newprecip3,newrel_humidity3,newwind_dir3,windspeed3,atmospherepressure3,newtemp4,newprecip4,newrel_humidity4,newwind_dir4,windspeed4,atmospherepressure4,newtemp5,newprecip5,newrel_humidity5,newwind_dir5,windspeed5,atmospherepressure5,newtemp6,newprecip6,newrel_humidity6,newwind_dir6,windspeed6,atmospherepressure6,newtemp7,newprecip7,newrel_humidity7,newwind_dir7,windspeed7,...,windspeed118,atmospherepressure118,newtemp119,newprecip119,newrel_humidity119,newwind_dir119,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_dir120,windspeed120,atmospherepressure120,max_temp,min_temp,mean_temp,std_temp,sum_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,sum_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,sum_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,sum_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,sum_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,sum_atmos_press,ptp_atmos_press
0,ID_train_0,C,45.126304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.745000,87.715833,19.533333,0.0,0.881500,329.178292,0.673333,87.750833,19.583333,0.000,0.874583,325.478885,0.855000,87.768333,27.208333,19.275000,22.299527,2.594011,356.792424,7.933333,0.000,0.0,0.000000,0.000000,0.000,0.000,0.891500,0.510818,0.758530,0.120234,12.136485,0.380682,345.726333,6.535299,209.574655,120.231802,3353.194474,339.191034,1.517500,0.162500,0.677348,0.394041,10.837576,1.355000,87.871667,87.614167,87.752434,0.070374,1404.038939,0.257500
1,ID_train_1,D,79.131702,22.533333,0.102,0.744583,281.664310,2.377500,90.320000,21.716667,0.0,0.808083,89.156293,1.126667,90.377500,20.833333,0.0,0.911167,81.968539,0.700833,90.440833,20.983333,0.000,0.916333,291.018632,0.341667,90.472500,20.875000,0.000,0.929750,279.391524,0.440833,90.454167,20.141667,0.0,0.954000,158.026892,0.380000,90.394167,19.375000,0.0,0.964083,248.316596,0.467500,90.333333,18.791667,0.034,0.970750,211.143363,0.4625,...,1.369167,90.325833,26.025000,0.0,0.649250,73.528733,1.475833,90.439167,21.450000,0.561,0.797333,296.967254,1.019167,90.529167,33.616667,17.983333,24.679063,4.266955,2986.166667,15.633333,0.561,0.0,0.007025,0.051706,0.850,0.561,0.979250,0.403083,0.758058,0.160952,91.725000,0.576167,357.730286,3.543198,177.109855,96.802929,21430.292415,354.187088,3.009167,0.248333,0.899208,0.539199,108.804167,2.760833,90.725000,90.056667,90.429924,0.156000,10942.020833,0.668333
2,ID_train_10,A,32.661304,28.975000,0.000,0.573333,NaN,NaN,88.551667,27.950000,0.0,0.597167,NaN,NaN,88.464167,29.600000,0.0,0.566833,NaN,NaN,88.319167,26.425000,0.102,0.627667,NaN,NaN,88.240000,22.091667,0.136,0.755417,NaN,NaN,88.230000,21.775000,0.0,0.777417,NaN,NaN,88.268333,22.333333,0.0,0.787000,NaN,NaN,88.370000,21.950000,2.160,0.831917,NaN,NaN,...,1.361667,88.515833,29.241667,0.0,0.459583,209.603600,1.481667,88.475833,30.091667,0.000,0.437917,250.105258,1.746667,88.415833,31.841667,18.458333,24.112317,3.776377,2893.478030,13.383333,7.804,0.0,0.151767,0.798401,18.212,7.804,0.930333,0.376750,0.712944,0.168129,85.553273,0.553583,357.265773,0.581510,201.373211,92.105195,15102.990803,356.684263,2.751667,0.331667,1.038656,0.585221,77.899167,2.420000,88.813333,87.982500,88.420931,0.179481,10610.511667,0.830833
3,ID_train_100,A,53.850238,22.966667,0.000,0.843083,300.085057,1.446667,88.615000,24.266667,0.0,0.790250,293.676960,1.192500,88.530833,25.275000,0.0,0.737500,294.517465,1.324167,88.400000,25.625000,0.000,0.728000,301.921417,1.544167,88.2

In [55]:
test.head()

,ID,location,target,newtemp0,newprecip0,newrel_humidity0,newwind_dir0,windspeed0,atmospherepressure0,newtemp1,newprecip1,newrel_humidity1,newwind_dir1,windspeed1,atmospherepressure1,newtemp2,newprecip2,newrel_humidity2,newwind_dir2,windspeed2,atmospherepressure2,newtemp3,newprecip3,newrel_humidity3,newwind_dir3,windspeed3,atmospherepressure3,newtemp4,newprecip4,newrel_humidity4,newwind_dir4,windspeed4,atmospherepressure4,newtemp5,newprecip5,newrel_humidity5,newwind_dir5,windspeed5,atmospherepressure5,newtemp6,newprecip6,newrel_humidity6,newwind_dir6,windspeed6,atmospherepressure6,newtemp7,newprecip7,newrel_humidity7,newwind_dir7,windspeed7,...,windspeed118,atmospherepressure118,newtemp119,newprecip119,newrel_humidity119,newwind_dir119,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_dir120,windspeed120,atmospherepressure120,max_temp,min_temp,mean_temp,std_temp,sum_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,sum_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,sum_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,sum_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,sum_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,sum_atmos_press,ptp_atmos_press
0,ID_test_0,C,NaN,23.966667,0.0,0.689833,290.135238,0.633333,87.643333,22.883333,0.0,0.722333,328.470113,0.725000,87.670000,22.283333,0.0,0.775583,323.573080,0.955000,87.705833,21.483333,0.0,0.838417,282.408045,0.872500,87.750833,20.908333,0.0,0.847500,278.460873,0.692500,87.791667,19.908333,0.0,0.859500,243.556262,0.347500,87.793333,19.141667,0.0,0.889917,63.683640,0.374167,87.761667,18.775000,0.0,0.918500,37.467025,0.430833,...,1.320000,87.570833,23.775000,0.000,0.694083,167.412340,1.289167,87.580000,22.766667,0.000,0.725250,163.667416,0.774167,87.625833,26.858333,17.050000,21.389532,2.731833,2588.133333,9.808333,5.728,0.0,0.204165,0.880254,24.704,5.728,0.999833,0.586083,0.842947,0.124179,101.153583,0.413750,357.223551,7.092226,231.104760,83.856214,27963.675994,350.131326,1.915833,0.171667,0.812555,0.392783,98.319167,1.744167,88.020833,87.474167,87.748506,0.132764,10617.569167,0.546667
1,ID_test_1,C,NaN,18.991667,0.0,0.999750,295.235054,0.640833,87.892500,19.708333,0.0,0.997250,307.656913,1.244167,87.956667,20.958333,0.0,0.956333,313.736024,1.284167,87.992500,22.783333,0.0,0.851583,311.518385,1.287500,87.992500,24.325000,0.0,0.777583,319.699892,1.310833,87.980833,25.016667,0.0,0.735833,324.856397,1.382500,87.928333,25.541667,0.0,0.704333,338.713614,0.941667,87.826667,26.925000,0.0,0.612583,63.036943,0.640833,...,0.765000,87.866667,20.075000,0.000,0.868917,174.138794,0.665833,87.920000,20.025000,0.000,0.886083,111.558812,0.311667,87.998333,26.991667,17.275000,21.302824,2.289268,2577.641667,9.716667,15.831,0.0,0.208719,1.517969,25.255,15.831,0.999750,0.583167,0.830610,0.106384,100.503833,0.416583,351.333059,6.290985,212.924101,91.254295,25763.816210,345.042074,2.099167,0.210000,0.852562,0.456733,103.160000,1.889167,88.181667,87.572500,87.936963,0.113756,10640.372500,0.609167
2,ID_test_10,D,NaN,25.116667,0.0,0.760167,77.061611,2.013333,90.340000,25.083333,0.0,0.733583,95.481250,1.302500,90.297500,24.408333,0.0,0.784750,61.444983,0.783333,90.305000,23.575000,0.0,0.825833,89.468836,0.530833,90.355833,22.350000,0.0,0.869750,298.164435,0.467500,90.442500,21.525000,0.0,0.906500,11.946559,0.409167,90.493333,21.266667,0.0,0.902667,262.764237,0.545000,90.536667,20.433333,0.0,0.890667,317.360242,0.515000,...,2.077500,90.414167,22.758333,0.136,0.892833,99.502531,2.285833,90.409167,20.608333,1.327,0.910167,68.393016,1.395000,90.464167,29.850000,17.108333,22.144421,2.943047,2679.475000,12.741667,1.327,0.0,0.040190,0.164596,4.863,1.327,0.986750,0.515167,0.868258,0.096427,105.059167,0.471583,356.508481,2.279896,167.731590,98.024235,20295.522338,354.228585,3.127500,0.237500,0.803140,0.569737,97.180000,2.890000,90.78

In [56]:
train.shape,test.shape

((15539, 765), (5035, 765))

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [58]:
test.drop("target",axis  = 1,inplace  = True)
train.shape,test.shape

((15539, 765), (5035, 764))

In [59]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()
lab.fit(train["location"])
train.location  = lab.transform(train.location)
test.location  = lab.transform(test.location)
train.target.min(),train.target.max()

(1.4526190476190477, 475.82)

In [30]:
!pip install catboost

     |████████████████████████████████| 64.8MB 64kB/s 


In [60]:
X=train.drop(["ID",'target'],axis =1)
y = train.target
test_id  = test['ID']

In [61]:
from sklearn.cluster import KMeans
X=X.fillna(-9999)

kmeans = KMeans(n_clusters=6).fit(X)
X['cluster']=kmeans.predict(X)
test=test.fillna(-9999)
test['cluster']=kmeans.predict(test.drop(["ID"],axis = 1))

In [62]:
air_temperature_filler = pd.DataFrame(X.groupby(['location','mean_wind_dir'])['mean_temp'].mean())
X['mean temp per month']=0
for i in range (len(X)):
  
  X['mean temp per month'][i]=air_temperature_filler.loc[(X['location'][i],X['mean_wind_dir'][i]), :]

air_temperature_filler = pd.DataFrame(test.groupby(['location','mean_wind_dir'])['mean_temp'].mean())

test['mean temp per month']=0
for i in range (len(test)):
  
  test['mean temp per month'][i]=air_temperature_filler.loc[(test['location'][i],test['mean_wind_dir'][i]), :]

In [63]:
p0=['newtemp0', 'newprecip0', 'newrel_humidity0', 'newwind_dir0',
       'windspeed0', 'atmospherepressure0']
p4=['newtemp30', 'newprecip30', 'newrel_humidity30', 'newwind_dir30',
       'windspeed30', 'atmospherepressure30']         
p1=['newtemp60', 'newprecip60', 'newrel_humidity60', 'newwind_dir60',
       'windspeed60', 'atmospherepressure60']  
p2=['newtemp90', 'newprecip90', 'newrel_humidity90', 'newwind_dir90',
       'windspeed90', 'atmospherepressure90']       
p3=['newtemp120', 'newprecip120', 'newrel_humidity120', 'newwind_dir120',
       'windspeed120', 'atmospherepressure120'] 
from sklearn.decomposition import PCA, FastICA
pca = PCA(random_state=42,n_components=1)
train_pca = pca.fit_transform(X[p0])
X['pca_0'] = train_pca[:,0]
test_pca = pca.fit_transform(test[p0])
test['pca_0'] = test_pca[:,0]
train_pca = pca.fit_transform(X[p1])
X['pca_1'] = train_pca[:,0]
test_pca = pca.fit_transform(test[p1])
test['pca_1'] = test_pca[:,0]
train_pca = pca.fit_transform(X[p2])
X['pca_2'] = train_pca[:,0]
test_pca = pca.fit_transform(test[p2])
test['pca_2'] = test_pca[:,0]

train_pca = pca.fit_transform(X[p3])
X['pca_3'] = train_pca[:,0]
test_pca = pca.fit_transform(test[p3])
test['pca_3'] = test_pca[:,0]

train_pca = pca.fit_transform(X[p4])
X['pca_4'] = train_pca[:,0]
test_pca = pca.fit_transform(test[p4])
test['pca_4'] = test_pca[:,0]

test=test.drop(columns=['pca_2','pca_4'])
X=X.drop(columns=['pca_4','pca_2'])

#generate 17 features of wind dir : u can run it and get the same data 

In [ ]:
import warnings
from pandas import read_csv
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
 
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	return best_cfg

def preidct (best,series,n):
        
        series = series.astype('float32')
        model = ARIMA(series, order=best)
        
      
        model_fit = model.fit(disp=0)
        output = model_fit.forecast(steps=n)
        yhat = output[0]


        return(yhat)

predictions=[]
nstep=17
p_values = [0, 1]
d_values = range(0, 1)
q_values = range(0, 1) 
for i in (test[winddir_features].T):       
       series=test[winddir_features].T[i]
       best=evaluate_models(series.values,p_values,d_values,q_values)
       print(best)

       yhat=preidct(best,series.values,nstep)
       for k in yhat[0:nstep] :
           predictions.append(k)

In [ ]:
winddirforecasttest=np.array(predictions)
winddirforecasttest=.reshape(5035,25)

In [ ]:
predictions=[]
nstep=17
p_values = [0, 1]
d_values = range(0, 1)
q_values = range(0, 1) 
for i in (X[winddir_features].T):       
       series=X[winddir_features].T[i]
       best=evaluate_models(series.values,p_values,d_values,q_values)
       print(best)

       yhat=preidct(best,series.values,nstep)
       for k in yhat[0:nstep] :
           predictions.append(k)

In [ ]:
winddirforecasttrain=np.array(predictions)
winddirforecasttrain=.reshape(15539,25)

In [ ]:
winddirforecasttrain=pd.DataFrame(winddirforecasttrain)
winddirforecasttest=pd.DataFrame(winddirforecasttest)

In [ ]:
winddirforecasttest.to_csv('winddirforecasttest.csv',index=False)
winddirforecasttrain.to_csv('winddirforecasttrain',index=False)

# **winddirforecasttrain & winddirforecasttest are used in the other kernels so please download it ,please upload it before you run other kernels **

In [64]:
X.shape,test.shape

((15539, 768), (5035, 769))

In [ ]:
((15539, 768), (5035, 769))

In [65]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    catf = np.where(X_train.dtypes != float)[0]
    m2  = CatBoostRegressor(   n_estimators=1000,eval_metric='RMSE',learning_rate=0.175, random_seed= 1234, use_best_model=True)
    m2.fit(X_train,y_train,cat_features=catf,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test.drop(["ID"],axis = 1))
    y_pred_totcb2.append(p2)
np.mean(errcb2)               #25.88---§37.96

0:	learn: 40.5922225	test: 40.5922225	test1: 41.4069704	best: 41.4069704 (0)	total: 398ms	remaining: 6m 37s
100:	learn: 23.6894430	test: 23.6894430	test1: 27.8916600	best: 27.8854686 (98)	total: 24.1s	remaining: 3m 34s
200:	learn: 19.7755645	test: 19.7755645	test1: 26.6203587	best: 26.6077002 (199)	total: 47.4s	remaining: 3m 8s
300:	learn: 16.9072691	test: 16.9072691	test1: 26.2804239	best: 26.2194317 (288)	total: 1m 10s	remaining: 2m 43s
400:	learn: 14.5697229	test: 14.5697229	test1: 26.0156032	best: 26.0156032 (400)	total: 1m 36s	remaining: 2m 24s
500:	learn: 12.8122542	test: 12.8122542	test1: 25.9068808	best: 25.9013278 (491)	total: 1m 59s	remaining: 1m 59s
600:	learn: 11.4520416	test: 11.4539961	test1: 25.7876493	best: 25.7409566 (576)	total: 2m 23s	remaining: 1m 35s
700:	learn: 10.1354188	test: 10.1394665	test1: 25.6139615	best: 25.6139615 (700)	total: 2m 46s	remaining: 1m 11s
800:	learn: 9.0415367	test: 9.0458813	test1: 25.5600453	best: 25.5419186 (798)	total: 3m 10s	remaining: 4

25.001169841467878

In [66]:
np.mean(errcb2)
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]
sub.target.min(),sub.target.max()

(13.538513232291512, 236.82383974540963)

In [67]:
sub.to_csv("sub1 catboost+mean.csv",index  = False)
from google.colab import files
files.download("sub1 catboost+mean.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# LGBM

In [68]:
import lightgbm as lgb
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.1,
    'num_leaves': 200,  
    'max_depth': 9,  
    'min_child_samples': 200,   
    'subsample': 0.7,  
    'subsample_freq': 1,  
    'colsample_bytree': 0.7,  
    'min_child_weight': 0,  
    'subsample_for_bin': 200000,  
    'min_split_gain': 0,  
    'reg_alpha': 0,  
    'reg_lambda': 0,  
    'nthread': -1,
    'verbose': 5,
    'n_estimators':10000,
#     'scale_pos_weight'
    }

model = lgb.LGBMRegressor(**params)

In [69]:
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=5)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = model
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test.drop(["ID"],axis = 1))
    y_pred_totcb2.append(p2)
np.mean(errcb2)

Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 23.4408	valid_1's rmse: 28.2851
[200]	training's rmse: 19.5167	valid_1's rmse: 27.3921
[300]	training's rmse: 16.718	valid_1's rmse: 26.9175
[400]	training's rmse: 14.5859	valid_1's rmse: 26.6435
[500]	training's rmse: 12.765	valid_1's rmse: 26.442
[600]	training's rmse: 11.2676	valid_1's rmse: 26.2943
[700]	training's rmse: 10.0661	valid_1's rmse: 26.1859
[800]	training's rmse: 8.94169	valid_1's rmse: 26.1166
[900]	training's rmse: 7.96012	valid_1's rmse: 26.0687
[1000]	training's rmse: 7.23242	valid_1's rmse: 26.0069
[1100]	training's rmse: 6.51877	valid_1's rmse: 25.9626
[1200]	training's rmse: 5.92853	valid_1's rmse: 25.9219
[1300]	training's rmse: 5.38788	valid_1's rmse: 25.8761
[1400]	training's rmse: 4.88144	valid_1's rmse: 25.8628
[1500]	training's rmse: 4.43245	valid_1's rmse: 25.8431
[1600]	training's rmse: 4.06741	valid_1's rmse: 25.8017
[1700]	training's rmse: 3.73537	valid_1's rmse: 25.77

25.384363058006922

In [70]:
np.mean(errcb2)
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub2 = pd.DataFrame(data=d)
sub2 = sub2[['ID', 'target']]
sub2.target.min(),sub2.target.max()

(4.597949962437147, 228.92226767888513)

In [71]:
sub2.to_csv("sub2.csv",index  = False)
from google.colab import files
files.download("sub2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Blend

In [72]:
submission1=sub.copy()
submission1["target"]=sub["target"]*0.6+0.4*sub2["target"]  
submission1.to_csv("submission1.csv",index  = False)
files.download("submission1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>